In [1]:
import pandas as pd
import numpy as np
import os
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
import scipy
from sklearn import linear_model, metrics, pipeline, preprocessing
import imblearn
pd.set_option('display.max_columns', None)

In [2]:
os.getcwd()

'C:\\Users\\RJP1F47\\Fraud Project'

In [3]:
import pandas as pd
import random
p = 0.7  # 1% of the lines
#### keep the header, then take only 1% of lines
#### if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         'tmx_data_jan_2021.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

In [4]:
#for reading full df
#df = pd.read_csv('tmx_data_jan_2021.csv')

In [5]:
df.columns

Index(['Event Time', 'Event Type', 'Risk Rating', 'Session ID',
       'LexID Digital Trust Score Rating', 'Review Status',
       'Final Review Status', 'Smart ID', 'Smart ID First Seen',
       'Smart Learning Summary Reasons', 'DNS IP', 'True IP', 'Input IP',
       'DNS IP City', 'Input IP City', 'True IP City', 'Device Model',
       'Challenger Risk Rating', 'Smart ID Local Trust Tags',
       'TMX Summary Reasons', 'TMX Reasons', 'Smart Learning Reasons',
       'Reasons', 'LexID Digital Reasons', 'True IP Country', 'True IP ISP',
       'Account Login First Seen', 'Smart ID Last Event',
       'LexID Digital Trust Score', 'Smart ID Match Result', 'Input IP ISP',
       'Account Email First Seen', 'Exact ID First Seen', 'OS', 'OS Anomaly',
       'Smart Learning P Score', 'Agent Connection Type', 'Jailbreak or Root',
       'Cookies Enabled', 'True IP Local Trust Tags', 'Variables',
       'Smart Learning Variables', 'Input IP Geo'],
      dtype='object')

In [6]:
#Supplemental Fraud Positive Data
df_fraud_supplemental = pd.read_csv('tmx_data_feb_2021_supp.csv')


In [7]:
df_fraud_supplemental.head()

,Event Time,Event Type,Risk Rating,Session ID,LexID Digital Trust Score Rating,Review Status,Final Review Status,Smart ID,Smart ID First Seen,Smart Learning Summary Reasons,DNS IP,True IP,Input IP,DNS IP City,Input IP City,True IP City,Device Model,Challenger Risk Rating,Smart ID Local Trust Tags,TMX Summary Reasons,TMX Reasons,Smart Learning Reasons,Reasons,LexID Digital Reasons,True IP Country,True IP ISP,Account Login First Seen,Smart ID Last Event,LexID Digital Trust Score,Smart ID Match Result,Input IP ISP,Account Email First Seen,Exact ID First Seen,OS,OS Anomaly,Smart Learning P Score,Agent Connection Type,Jailbreak or Root,Cookies Enabled,True IP Local Trust Tags,Variables,Smart Learning Variables,Input IP Geo,Policy Score
0,2021-02-09 04:59:17.597 UTC,login,trusted,5cdf61f5-6461-4950-aa02-439b4f0af8cb,very_high,pass,reject,a2c57849d73c4fbabd17c401d12ebda1,2020-07-05 00:00:00.0 UTC,NaN,172.253.1.4,NaN,198.199.92.159,los angeles,san francisco,NaN,NaN,trusted,"{""_CHALLENGE_PASSED"",""_FRAUD_PROB""}","{""Device_Negative_History"",""Identity_Negative_...","{""_IP_GBL_VEL_120_120_120_120"",""_IP_GBL_AGE_GT...","{""_Email_FRS_accept_local_1_3"",""_Rejected_Inpu...","{""NON_home_depot_ISP"",""Nonsense_Email"",""Potent...",NaN,NaN,NaN,NaN,2021-02-09 00:00:00.0 UTC,93.0,success,digitalocean llc,2019-09-07 00:00:00.0 UTC,2021-01-29 00:00:00.0 UTC,Windows,NaN,0.465,NaN,NaN,False,NaN,"{""_accemail_gbl_velocity_hour"":""4"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",US,-16
1,2021-02-09 04:59:15.309 UTC,login,high,e0fc9951-bab7-4d13-a809-6e792c08d5f5,high,review,reject,eace26ccb5224202a89df2e80f600995,2020-10-08 00:00:00.0 UTC,NaN,49.44.189.190,157.49.247.180,NaN,gurgaon,NaN,bengaluru,NaN,high,"{""_CHALLENGE_PASSED"",""_FRAUD_PROB""}","{""Device_Negative_History"",""Identity_Negative_...","{""_IP_GBL_VEL_63_63_63_73"",""_IP_GBL_AGE_GT_3MT...","{""_ACC_EMAIL_SMART_ID_GBL_PAIR_LAST_TXN_1MIN"",...","{""Nonsense_Email"",""Potential_Machine_Generated...","{""exact_id_velocity_high"",""exact_id_gbl_veloci...",india,reliance jio infocomm limited,NaN,2021-02-09 00:00:00.0 UTC,81.0,success,NaN,2020-06-24 00:00:00.0 UTC,2020-10-09 00:00:00.0 UTC,Windows,NaN,0.869,NaN,NaN,False,NaN,"{""_accemail_gbl_velocity_hour"":""61"",""_accemail...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""4"",""_s...",NaN,-35
2,2021-02-09 04:59:13.777 UTC,login,high,e0fc9951-bab7-4d13-a809-6e792c08d5f5,high,review,reject,eace26ccb5224202a89df2e80f600995,2020-10-08 00:00:00.0 UTC,NaN,49.44.189.190,157.49.247.180,NaN,gurgaon,NaN,bengaluru,NaN,high,"{""_CHALLENGE_PASSED"",""_FRAUD_PROB""}","{""Device_Negative_History"",""Identity_Negative_...","{""_IP_GBL_VEL_62_62_62_72"",""_IP_GBL_AGE_GT_3MT...","{""_ACC_EMAIL_SMART_ID_GBL_PAIR_LAST_TXN_1MIN"",...","{""Nonsense_Email"",""Potential_Machine_Generated...","{""exact_id_velocity_high"",""exact_id_gbl_veloci...",india,reliance jio infocomm limited,NaN,2021-02-09 00:00:00.0 UTC,80.0,success,NaN,2020-06-24 00:00:00.0 UTC,2020-10-09 00:00:00.0 UTC,Windows,NaN,0.904,NaN,NaN,False,NaN,"{""_accemail_gbl_velocity_hour"":""60"",""_accemail...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""4"",""_s...",NaN,-34
3,2021-02-09 04:59:10.405 UTC,login,high,11a70f75-8974-4643-ab9b-ea483a8e534a,high,review,reject,eace26ccb5224202a89df2e80f600995,2020-10-08 00:00:00.0 UTC,NaN,49.44.189.189,157.49.247.180,NaN,gurgaon,NaN,bengaluru,NaN,high,"{""_CHALLENGE_PASSED"",""_FRAUD_PROB""}","{""Device_Negative_History"",""Identity_Negative_...","{""_IP_GBL_VEL_61_61_61_71"",""_IP_GBL_AGE_GT_3MT...","{""_ACC_EMAIL_SMART_ID_GBL_PAIR_LAST_TXN_1MIN"",...","{""Nonsense_Email"",""Potential_Machine_Generated...","{""exact_id_velocity_high"",""exact_id_gbl_veloci...",india,reliance jio infocomm limited,NaN,2021-02-09 00:00:00.0 UTC,80.0,success,NaN,2020-06-24 00:00:00.0 UTC,2020-10-09 00:00:00.0 UTC,Windows,NaN,0.899,NaN,NaN,False,NaN,"{""_accemail_gbl_velocity_hour"":""59"",""_accemail...","{""_sl_v0_dist_accemail_per_sid_lcl_wk""

In [8]:
df.shape

(1115222, 43)

In [9]:
df_fraud_supplemental = df_fraud_supplemental[df.columns]


In [10]:
df_fraud_supplemental.tail()

,Event Time,Event Type,Risk Rating,Session ID,LexID Digital Trust Score Rating,Review Status,Final Review Status,Smart ID,Smart ID First Seen,Smart Learning Summary Reasons,DNS IP,True IP,Input IP,DNS IP City,Input IP City,True IP City,Device Model,Challenger Risk Rating,Smart ID Local Trust Tags,TMX Summary Reasons,TMX Reasons,Smart Learning Reasons,Reasons,LexID Digital Reasons,True IP Country,True IP ISP,Account Login First Seen,Smart ID Last Event,LexID Digital Trust Score,Smart ID Match Result,Input IP ISP,Account Email First Seen,Exact ID First Seen,OS,OS Anomaly,Smart Learning P Score,Agent Connection Type,Jailbreak or Root,Cookies Enabled,True IP Local Trust Tags,Variables,Smart Learning Variables,Input IP Geo
14438,2021-02-01 05:12:48.294 UTC,login,low,ad13619e-7f2d-4eac-aee9-79b6d3a5cadf,very_high,pass,reject,d1c3b9412d2c448ab2007eb160a2cadc,2021-01-23 00:00:00.0 UTC,NaN,184.178.228.16,68.8.74.124,NaN,los angeles,NaN,san diego,NaN,low,NaN,"{""Identity_Negative_History"",""Level_1_Link_Acc...","{""_IP_GBL_VEL_1_1_5_5"",""_IP_GBL_AGE_GT_3MTHS"",...","{""_Email_FRS_accept_local_1_3"",""_SMART_ID_GBL_...","{""Nonsense_Email"",""Potential_Machine_Generated...",NaN,united states,cox communications inc.,NaN,2021-02-01 00:00:00.0 UTC,84.0,success,NaN,2020-04-28 00:00:00.0 UTC,2021-01-23 00:00:00.0 UTC,iOS,NaN,0.762,NaN,NaN,False,NaN,"{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",NaN
14439,2021-02-01 05:07:09.812 UTC,login,trusted,30d5e6fa-9a81-447d-a6c1-0c8bd7ffaa70,high,pass,reject,a2c57849d73c4fbabd17c401d12ebda1,2020-07-05 00:00:00.0 UTC,NaN,172.253.0.3,198.199.94.250,198.199.94.250,los angeles,san francisco,san francisco,NaN,trusted,"{""_CHALLENGE_PASSED""}","{""Device_Negative_History"",""Identity_Negative_...","{""_IP_GBL_VEL_15_19_71_120"",""_IP_GBL_AGE_GT_3M...","{""_Email_FRS_accept_local_1_3"",""_Rejected_True...","{""NON_home_depot_ISP"",""Nonsense_Email"",""Potent...",NaN,united states,digitalocean llc,NaN,2021-02-01 00:00:00.0 UTC,81.0,success,digitalocean llc,2019-09-07 00:00:00.0 UTC,2021-01-29 00:00:00.0 UTC,Windows,NaN,0.513,NaN,NaN,False,"{""_FRAUD_PROB""}","{""_accemail_gbl_velocity_hour"":""1"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""2"",""_s...",US
14440,2021-02-01 05:05:17.460 UTC,login,low,5b799074fdffca107852dfa4a8306d64,high,pass,reject,88b47bde036c4ad4978ebae4e034385e,2020-10-07 00:00:00.0 UTC,NaN,107.77.165.245,107.77.165.83,107.77.165.19,san bernardino,san bernardino,san bernardino,moto g(7) supra,low,NaN,"{""Level_1_Link_Accept""}","{""_IP_GBL_VEL_9_120_120_120"",""_IP_GBL_AGE_GT_3...","{""_Email_FRS_accept_local_1_3"",""_EXACT_ID_LCL_...","{""NON_home_depot_ISP"",""Nonsense_Email"",""Potent...",NaN,united states,at&t mobility llc,NaN,2021-02-01 00:00:00.0 UTC,79.0,success,at&t mobility llc,2020-08-11 00:00:00.0 UTC,2020-10-07 00:00:00.0 UTC,Android,NaN,0.536,cellular,0.0,False,"{""_CHALLENGE_PASSED"",""_FRAUD_PROB""}","{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",US
14441,2021-02-01 05:03:23.188 UTC,login,low,bf28b274-88f7-46f6-ab0c-ff7df770b7e7,low,pass,reject,73c65a9b455448b8b8ab8d51dbf552a1,2020-10-17 00:00:00.0 UTC,NaN,NaN,212.102.46.22,NaN,NaN,NaN,seattle,NaN,low,"{""_FRAUD_PROB""}","{""Identity_Negative_History"",""Identity_Spoofing""}","{""_IP_GBL_VEL_5_120_120_120"",""_IP_GBL_AGE_GT_3...","{""_ACC_EMAIL_SMART_ID_LCL_PAIR_AGE_GT_MTH"",""_R...","{""Nonsense_Email"",""Potential_Machine_Generated...",NaN,united states,datacamp limited,NaN,2021-02-01 00:00:00.0 UTC,55.0,success,NaN,2020-08-25 00:00:00.0 UTC,2020-10-17 00:00:00.0 UTC,Windows,False,0.899,NaN,NaN,False,"{""_FRAUD_PROB"",""_WATCH""}","{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",NaN
14442,2021-02-01 05:00:43.415 UTC,login,low,1ecfae90-7bf2-49d9-ba55-a2e00937e37a,high,pass,reject,475f85eb8cd64fb5bf94d9397f96f12

In [11]:
df = pd.concat([df,df_fraud_supplemental], sort=False)
df = df.reset_index()

In [12]:
df.shape

(1129665, 44)

In [13]:
df.tail()

,index,Event Time,Event Type,Risk Rating,Session ID,LexID Digital Trust Score Rating,Review Status,Final Review Status,Smart ID,Smart ID First Seen,Smart Learning Summary Reasons,DNS IP,True IP,Input IP,DNS IP City,Input IP City,True IP City,Device Model,Challenger Risk Rating,Smart ID Local Trust Tags,TMX Summary Reasons,TMX Reasons,Smart Learning Reasons,Reasons,LexID Digital Reasons,True IP Country,True IP ISP,Account Login First Seen,Smart ID Last Event,LexID Digital Trust Score,Smart ID Match Result,Input IP ISP,Account Email First Seen,Exact ID First Seen,OS,OS Anomaly,Smart Learning P Score,Agent Connection Type,Jailbreak or Root,Cookies Enabled,True IP Local Trust Tags,Variables,Smart Learning Variables,Input IP Geo
1129660,14438,2021-02-01 05:12:48.294 UTC,login,low,ad13619e-7f2d-4eac-aee9-79b6d3a5cadf,very_high,pass,reject,d1c3b9412d2c448ab2007eb160a2cadc,2021-01-23 00:00:00.0 UTC,NaN,184.178.228.16,68.8.74.124,NaN,los angeles,NaN,san diego,NaN,low,NaN,"{""Identity_Negative_History"",""Level_1_Link_Acc...","{""_IP_GBL_VEL_1_1_5_5"",""_IP_GBL_AGE_GT_3MTHS"",...","{""_Email_FRS_accept_local_1_3"",""_SMART_ID_GBL_...","{""Nonsense_Email"",""Potential_Machine_Generated...",NaN,united states,cox communications inc.,NaN,2021-02-01 00:00:00.0 UTC,84.0,success,NaN,2020-04-28 00:00:00.0 UTC,2021-01-23 00:00:00.0 UTC,iOS,NaN,0.762,NaN,NaN,False,NaN,"{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",NaN
1129661,14439,2021-02-01 05:07:09.812 UTC,login,trusted,30d5e6fa-9a81-447d-a6c1-0c8bd7ffaa70,high,pass,reject,a2c57849d73c4fbabd17c401d12ebda1,2020-07-05 00:00:00.0 UTC,NaN,172.253.0.3,198.199.94.250,198.199.94.250,los angeles,san francisco,san francisco,NaN,trusted,"{""_CHALLENGE_PASSED""}","{""Device_Negative_History"",""Identity_Negative_...","{""_IP_GBL_VEL_15_19_71_120"",""_IP_GBL_AGE_GT_3M...","{""_Email_FRS_accept_local_1_3"",""_Rejected_True...","{""NON_home_depot_ISP"",""Nonsense_Email"",""Potent...",NaN,united states,digitalocean llc,NaN,2021-02-01 00:00:00.0 UTC,81.0,success,digitalocean llc,2019-09-07 00:00:00.0 UTC,2021-01-29 00:00:00.0 UTC,Windows,NaN,0.513,NaN,NaN,False,"{""_FRAUD_PROB""}","{""_accemail_gbl_velocity_hour"":""1"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""2"",""_s...",US
1129662,14440,2021-02-01 05:05:17.460 UTC,login,low,5b799074fdffca107852dfa4a8306d64,high,pass,reject,88b47bde036c4ad4978ebae4e034385e,2020-10-07 00:00:00.0 UTC,NaN,107.77.165.245,107.77.165.83,107.77.165.19,san bernardino,san bernardino,san bernardino,moto g(7) supra,low,NaN,"{""Level_1_Link_Accept""}","{""_IP_GBL_VEL_9_120_120_120"",""_IP_GBL_AGE_GT_3...","{""_Email_FRS_accept_local_1_3"",""_EXACT_ID_LCL_...","{""NON_home_depot_ISP"",""Nonsense_Email"",""Potent...",NaN,united states,at&t mobility llc,NaN,2021-02-01 00:00:00.0 UTC,79.0,success,at&t mobility llc,2020-08-11 00:00:00.0 UTC,2020-10-07 00:00:00.0 UTC,Android,NaN,0.536,cellular,0.0,False,"{""_CHALLENGE_PASSED"",""_FRAUD_PROB""}","{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",US
1129663,14441,2021-02-01 05:03:23.188 UTC,login,low,bf28b274-88f7-46f6-ab0c-ff7df770b7e7,low,pass,reject,73c65a9b455448b8b8ab8d51dbf552a1,2020-10-17 00:00:00.0 UTC,NaN,NaN,212.102.46.22,NaN,NaN,NaN,seattle,NaN,low,"{""_FRAUD_PROB""}","{""Identity_Negative_History"",""Identity_Spoofing""}","{""_IP_GBL_VEL_5_120_120_120"",""_IP_GBL_AGE_GT_3...","{""_ACC_EMAIL_SMART_ID_LCL_PAIR_AGE_GT_MTH"",""_R...","{""Nonsense_Email"",""Potential_Machine_Generated...",NaN,united states,datacamp limited,NaN,2021-02-01 00:00:00.0 UTC,55.0,success,NaN,2020-08-25 00:00:00.0 UTC,2020-10-17 00:00:00.0 UTC,Windows,False,0.899,NaN,NaN,False,"{""_FRAUD_PROB"",""_WATCH""}","{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",NaN
1129664,14442,2021-02-01 05:00:43.415 UTC,login,low,1ecfae90-7bf2-49d9-ba55-a2e00937e37a,hi

In [14]:
del df_fraud_supplemental

In [15]:
df.head()

,index,Event Time,Event Type,Risk Rating,Session ID,LexID Digital Trust Score Rating,Review Status,Final Review Status,Smart ID,Smart ID First Seen,Smart Learning Summary Reasons,DNS IP,True IP,Input IP,DNS IP City,Input IP City,True IP City,Device Model,Challenger Risk Rating,Smart ID Local Trust Tags,TMX Summary Reasons,TMX Reasons,Smart Learning Reasons,Reasons,LexID Digital Reasons,True IP Country,True IP ISP,Account Login First Seen,Smart ID Last Event,LexID Digital Trust Score,Smart ID Match Result,Input IP ISP,Account Email First Seen,Exact ID First Seen,OS,OS Anomaly,Smart Learning P Score,Agent Connection Type,Jailbreak or Root,Cookies Enabled,True IP Local Trust Tags,Variables,Smart Learning Variables,Input IP Geo
0,0,2021-01-09 16:59:59.931 UTC,login,low,61c6888f-c6c3-4874-82f0-6aef818f037b,very_high,pass,NaN,7c08a992cd514b6c86b138c1ae434ff3,2020-09-04 00:00:00.0 UTC,NaN,205.171.176.193,75.167.236.55,75.167.236.55,phoenix,phoenix,phoenix,NaN,low,NaN,NaN,"{""_IP_GBL_VEL_7_7_9_13"",""_IP_GBL_AGE_GT_3MTHS""...","{""_ACC_EMAIL_SMART_ID_GBL_PAIR_LAST_TXN_1MIN"",...","{""Dup_Session_ID_5x"",""2Dim_Good_Persona_Email_...",NaN,united states,centurylink communications llc,NaN,2021-01-09 00:00:00.0 UTC,84.0,success,centurylink communications llc,2020-03-30 00:00:00.0 UTC,2020-09-09 00:00:00.0 UTC,iOS,NaN,0.630,NaN,NaN,True,NaN,"{""_accemail_gbl_velocity_hour"":""2"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""3"",""_s...",US
1,1,2021-01-09 16:59:59.919 UTC,login,low,55e9e632ebed4ed7b51b63c37c4fe2c8,very_high,pass,NaN,23d8ace2244145dd96eef34bdde80725,2019-02-09 00:00:00.0 UTC,NaN,12.121.90.149,99.41.196.155,99.41.196.155,houston,san antonio,san antonio,"iPhone13,3",low,"{""_CHALLENGE_PASSED""}","{""Identity_Negative_History""}","{""_IP_GBL_VEL_1_1_24_70"",""_IP_GBL_AGE_GT_3MTHS...","{""_AGENT_PUBLICKEY_HASH_LCL_AGE_GT_2MTHS"",""_EX...","{""Nonsense_Email"",""Potential_Machine_Generated...",NaN,united states,att services inc,NaN,2021-01-09 00:00:00.0 UTC,96.0,success,att services inc,2015-10-20 00:00:00.0 UTC,2019-02-09 00:00:00.0 UTC,iOS,NaN,0.668,wifi,0.0,True,NaN,"{""_accemail_gbl_velocity_hour"":""0"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",US
2,2,2021-01-09 16:59:59.881 UTC,login,low,1a7f42be-7f5b-4edc-8906-7690b733d0dc,very_high,pass,NaN,fd7c1e809b764acca97e8cf603e2169e,2020-12-27 00:00:00.0 UTC,NaN,NaN,71.232.119.55,71.232.119.55,NaN,somerville,somerville,NaN,low,NaN,"{""Level_1_Link_Accept""}","{""_IP_GBL_VEL_3_5_5_15"",""_IP_GBL_AGE_GT_3MTHS""...","{""_IP_LCL_AGE_GT_2MTHS"",""_SMART_ID_GBL_AGE_GT_...","{""5x_SmartID_per_DID_Mth_Gbl"",""New_SmartID_LT_...",NaN,united states,comcast cable communications llc,NaN,2021-01-09 00:00:00.0 UTC,97.0,success,comcast cable communications llc,2019-03-31 00:00:00.0 UTC,2020-12-27 00:00:00.0 UTC,Windows,NaN,0.512,NaN,NaN,True,NaN,"{""_accemail_gbl_velocity_hour"":""2"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",US
3,3,2021-01-09 16:59:59.702 UTC,login,low,04738193-f5c6-4073-91a4-5082c4f4cd62,very_high,pass,NaN,b3a405a353ad4633a1d7eff3106be744,2021-01-09 00:00:00.0 UTC,NaN,24.175.34.40,70.118.139.11,70.118.139.11,san antonio,killeen,killeen,NaN,low,NaN,NaN,"{""_IP_GBL_VEL_4_4_7_9"",""_IP_GBL_AGE_GT_3MTHS"",...","{""_IP_LCL_AGE_GT_2MTHS"",""_AGENT_PUBLICKEY_HASH...","{""Nonsense_Email"",""Potential_Machine_Generated...",NaN,united states,charter communications inc,NaN,2021-01-09 00:00:00.0 UTC,93.0,success,charter communications inc,2017-12-05 00:00:00.0 UTC,2021-01-09 00:00:00.0 UTC,iOS,NaN,0.113,NaN,NaN,True,NaN,"{""_accemail_gbl_velocity_hour"":""3"",""_accemail_...","{""_sl_v0_dist_accemail_per_sid_lcl_wk"":""1"",""_s...",US
4,4,2021-01-09 16:59:59.667 UTC,login,low,e1b3e0f484924373af6d1660aa40eda3,very_high,pass,NaN,9f4cce0c68254bfeba73a2a8d6d681a9,2019-02-18 00:00:00.0 UTC,NaN,76.96.47.201,73.78.198.218,73.78.198.218,denver,englewood,englewood,"iPhone12,3",low,NaN,"{""Level_1_Link_Accept""}","{""_IP

## Data Cleaning
-set up a categorical_features list of features that need work on. Anything left over at the end should be numerical data (which our sampling method will make into objects so it is not as easy to graph later)
-we then undersample to match the minority class size (1 which is faur)
-we then create our expand_list which is a list of categorical features that have multi labels at each data point
-we then filter to work out some of the extra string data attached and allow us to easily create dummies

In [16]:
#only login events
df = df[df['Event Type'] == 'login']

#setup target variable
y = df['Final Review Status'].copy()
y[:] = 0
#target is anything that wasn't reviewed by the policy by came back as fraud
y[(df['Final Review Status'] == 'reject') & ((df['Review Status'] != 'review') | (df['Review Status'] != 'reject'))] = 1


#variables that are not important or have too much power #DOUBLE CHECK LATER
df = df.drop(['True IP City', 'True IP ISP','True IP', 'DNS IP', 'Smart ID','Input IP','Variables', 'TMX Summary Reasons','Smart Learning Summary Reasons','Session ID','Event Type','Review Status', 'Smart Learning Variables', 'True IP Local Trust Tags', 'Event Time', 'Smart ID First Seen', 'Account Login First Seen', 'Account Email First Seen', 'Smart ID Last Event', 'Exact ID First Seen'],axis = 1)

#fill categorical missing variables with "Missing"
categorical_features = list(df.select_dtypes(include = ["object"]).columns)
df[categorical_features] = df[categorical_features].fillna('Missing')
y = y.fillna('Missing')

df = df.drop(['Final Review Status'],axis=1)
categorical_features.remove('Final Review Status')

#y data cleanup
#y_cleanup = {'Missing': 0, 'accept':0, 'reject':1, 'Presumed Fraud':1}
#y = pd.DataFrame(y.replace(y_cleanup), dtype=bool)

In [17]:
train_labels = list(df.columns)

In [18]:
#undersample = imblearn.under_sampling.RandomUnderSampler(sampling_strategy='not minority')
#undersample = imblearn.under_sampling.RandomUnderSampler(sampling_strategy=0.1)
#df, y = undersample.fit_resample(df, y)

In [19]:
df = pd.DataFrame(df,columns=train_labels)

In [20]:
df.shape

(1043677, 23)

In [21]:
expand_list = ['Smart ID Local Trust Tags','TMX Reasons','Smart Learning Reasons','Reasons']

#filter through to remove extra data and clean up
def extract_and_filter(row):
    words = "".join(i for i in row[col] if i not in ['{', "}", "\"", ] )
    row[col] = words
    #print(row)
    return row[col]
   

In [22]:
#make sure all columns are strings
df[expand_list] = df[expand_list].astype(str)

In [23]:
df.shape

(1043677, 23)

In [24]:
#filter through to remove extra data and clean up
def extract_and_filter(row, column):
    words = "".join(i for i in row[column] if i not in ['{', "}", "\"", ] )
    row[column] = words
    #print(row)
    return row[column]
   

In [25]:
#I could use map instead of looping through and setting the column and then recalling that in the function
for j in expand_list:
    #df[j] = df.apply(lambda row: extract_and_filter(row[j]), axis=1)
    col = j
    #df1[j] = df1.apply(lambda row: extract_and_filter(row[j]), axis=1)
    df[j] = df.apply(extract_and_filter, column = col, axis=1)
    #df[j] = df[j].apply(extract_and_filter, axis=1)
    

In [26]:
df.shape

(1043677, 23)

In [27]:
#hashing out since we are sampling at import now
#df_multi_labels = df.sample(frac=0.5, replace=True, random_state=1).copy()
#df_multi_labels = df.copy()

#del df

## Feature Engineering

#### Change Multi Label Categorical Variables list variables
1.create a dummies sparse matrix to store all the dummy variable data

2.run multilabelbinarizer on each multi label categorical feature and merge the output to our df

3.drop original multi label categorical features from df

4.remove expand list from categorical variables list
#### Change Normal Categorical variables

3. do one DictVect on the rest of the categorical variables

### Multi Label Binarizer
-can save space by dropping as we go in the binarizer instead of after

In [28]:
mlb = MultiLabelBinarizer(sparse_output=True) #will output csr format


In [29]:
dummies = scipy.sparse.csr.csr_matrix((df.shape[0],0))

In [30]:
#create this variable to store our column names as we go
columns = list()

In [31]:
expand_list

['Smart ID Local Trust Tags',
 'TMX Reasons',
 'Smart Learning Reasons',
 'Reasons']

In [32]:
##### sparse matrix version
for i in expand_list:
    #creates each observation into a list of values contrained within one larger list for the column
    #holding_df = df[i].str.split(',')#.values.tolist() or to_array()
    #converts into a dataframe of 1's and 0's
    label_df_sparse = scipy.sparse.csr.csr_matrix(mlb.fit_transform(df[i].str.split(',')))
    df = df.drop([i], axis=1)
    
    #capturing the labels
    columns.extend(mlb.classes_)
    #column_dict = dict(enumerate(columns))
    #print(mlb.classes_)

    print("feature: ", i, "\nUnique Values: ", label_df_sparse.shape[1])

    dummies = scipy.sparse.hstack([dummies, label_df_sparse])

feature:  Smart ID Local Trust Tags 
Unique Values:  5
feature:  TMX Reasons 
Unique Values:  592098
feature:  Smart Learning Reasons 
Unique Values:  117
feature:  Reasons 
Unique Values:  256


##### Non sparse matrix version
for i in expand_list:
    holding_df = df[i].str.split(',')#.values.tolist() or to_array()
    label_df = pd.DataFrame(mlb.fit_transform(holding_df), columns=mlb.classes_, index=holding_df.index)
    print("feature: ", i, "\nUnique Values: ", label_df.shape[1])
    #df_multi_labels.merge(label_df)
    df = pd.concat([df,label_df], axis=1)

In [33]:
#drop unneeded multi label columns now
#df = df.drop(expand_list, axis=1) #MOVED TO IN THE FOR LOOP NOW

In [34]:
df.head()

,index,Risk Rating,LexID Digital Trust Score Rating,DNS IP City,Input IP City,Device Model,Challenger Risk Rating,LexID Digital Reasons,True IP Country,LexID Digital Trust Score,Smart ID Match Result,Input IP ISP,OS,OS Anomaly,Smart Learning P Score,Agent Connection Type,Jailbreak or Root,Cookies Enabled,Input IP Geo
0,0,low,very_high,phoenix,phoenix,Missing,low,Missing,united states,84.0,success,centurylink communications llc,iOS,Missing,0.630,Missing,NaN,True,US
1,1,low,very_high,houston,san antonio,"iPhone13,3",low,Missing,united states,96.0,success,att services inc,iOS,Missing,0.668,wifi,0.0,True,US
2,2,low,very_high,Missing,somerville,Missing,low,Missing,united states,97.0,success,comcast cable communications llc,Windows,Missing,0.512,Missing,NaN,True,US
3,3,low,very_high,san antonio,killeen,Missing,low,Missing,united states,93.0,success,charter communications inc,iOS,Missing,0.113,Missing,NaN,True,US
4,4,low,very_high,denver,englewood,"iPhone12,3",low,Missing,united states,97.0,success,comcast cable communications llc,iOS,Missing,0.470,wifi,0.0,True,US


In [35]:
categorical_features = list(set(categorical_features) - set(expand_list))

### Date Handling
-subtract date features used from category_features list

### Single Label Binarizer with DictVect

In [36]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer

In [37]:
categorical_features

['OS Anomaly',
 'Agent Connection Type',
 'Cookies Enabled',
 'Smart ID Match Result',
 'Challenger Risk Rating',
 'LexID Digital Reasons',
 'Input IP Geo',
 'Device Model',
 'LexID Digital Trust Score Rating',
 'True IP Country',
 'Input IP City',
 'DNS IP City',
 'Input IP ISP',
 'Risk Rating',
 'OS']

In [38]:
#temporary
df[categorical_features].shape
df[categorical_features].isna().sum()
df[categorical_features].head()

,OS Anomaly,Agent Connection Type,Cookies Enabled,Smart ID Match Result,Challenger Risk Rating,LexID Digital Reasons,Input IP Geo,Device Model,LexID Digital Trust Score Rating,True IP Country,Input IP City,DNS IP City,Input IP ISP,Risk Rating,OS
0,Missing,Missing,True,success,low,Missing,US,Missing,very_high,united states,phoenix,phoenix,centurylink communications llc,low,iOS
1,Missing,wifi,True,success,low,Missing,US,"iPhone13,3",very_high,united states,san antonio,houston,att services inc,low,iOS
2,Missing,Missing,True,success,low,Missing,US,Missing,very_high,united states,somerville,Missing,comcast cable communications llc,low,Windows
3,Missing,Missing,True,success,low,Missing,US,Missing,very_high,united states,killeen,san antonio,charter communications inc,low,iOS
4,Missing,wifi,True,success,low,Missing,US,"iPhone12,3",very_high,united states,englewood,denver,comcast cable communications llc,low,iOS


In [39]:
#one hot encoding, merging dummy variables
enc = DictVectorizer(sparse = True)

enc_data = scipy.sparse.csr.csr_matrix(enc.fit_transform(df[categorical_features].T.to_dict().values()))

dummies = scipy.sparse.hstack([dummies, enc_data])

columns.extend(enc.get_feature_names())


df = df.drop(categorical_features, axis=1)
del enc_data

In [40]:
dummies.shape

(1043677, 610110)

In [41]:
len(columns)

610110

#### Cleanup of dummies before adding numerical data

In [42]:
dummies = scipy.sparse.csc.csc_matrix(dummies) #we want csc format to speed up column indexing
type(dummies)


scipy.sparse.csc.csc_matrix

In [43]:
def remove_low_freq_features(df, labels, min_frequency):
    to_keep = [i for i,j in enumerate(np.array(dummies.sum(axis=0))[0]) if j > min_frequency]
    df = df[:,to_keep]
    #labels = labels[to_keep]
    labels = [labels[index] for index in to_keep]
    return df, labels
    

In [44]:
#double check removal worked correctly
#np.array(dummies.sum(axis=0))[0][0:20]

In [45]:
#columns[0:20]

In [46]:
dummies, columns = remove_low_freq_features(dummies, columns, 5)

In [47]:
#columns[0:20]

In [48]:
dummies.shape

(1043677, 155507)

In [49]:
len(columns)

155507

### Numerical Data Setup
-make a dummy variable for the missing values

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043677 entries, 0 to 1129664
Data columns (total 4 columns):
index                        1043677 non-null int64
LexID Digital Trust Score    1036842 non-null float64
Smart Learning P Score       1041632 non-null float64
Jailbreak or Root            463624 non-null float64
dtypes: float64(3), int64(1)
memory usage: 79.8 MB


In [51]:
df = pd.DataFrame(df, dtype= float)

In [52]:
#df = df.fillna(df.mean())
df = df.fillna(df.median())

In [53]:
#adding our numerical column labels, before they get lost in scaling, to our list containing all the previous dummy variable column names
temp_cols = list(df.columns)
columns.extend(temp_cols)

In [54]:
df = preprocessing.StandardScaler().fit_transform(df)

In [55]:
dummies.shape

(1043677, 155507)

In [56]:
df = scipy.sparse.hstack([dummies, df])
del dummies

In [57]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                             y: 15.9 MiB
                       columns:  1.3 MiB
                            _7: 44.6 KiB
                           _13: 42.2 KiB
                           _10: 42.1 KiB
                           _15: 37.2 KiB
                            _5:  5.0 KiB
                           _34:  4.8 KiB
                           _38:  4.7 KiB
                          _i16:  1.3 KiB


In [58]:
df.shape

(1043677, 155511)

## Modeling
-test model on full dataset, will probably do even better

In [59]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import HuberRegressor

In [60]:
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=30, stratify=y)

In [61]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                             y: 15.9 MiB
                       y_train: 12.7 MiB
                        y_test:  3.2 MiB
                       columns:  1.3 MiB
                            _7: 44.6 KiB
                           _13: 42.2 KiB
                           _10: 42.1 KiB
                           _15: 37.2 KiB
                            _5:  5.0 KiB
                           _34:  4.8 KiB


In [62]:
#dir()

In [63]:
from sklearn.linear_model import Lasso

In [64]:
#model = LogisticRegression(solver='lbfgs', class_weight='balanced')
# define evaluation procedure
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
# evaluate model
#scores = cross_val_score(model, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
#print('Mean ROC AUC: %.3f' % mean(scores))

In [65]:
model = LogisticRegression(solver='liblinear', class_weight='balanced')
# define evaluation procedure
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
# evaluate model
scores = cross_val_score(model, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.988


In [66]:
model = LogisticRegression(solver='saga', class_weight='balanced', penalty='l1') #penalty='elasticnet') #need to install recent scikit learn
# define evaluation procedure
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=1)

tuning_params = {'C': [0.4,0.7,0.95]}

gs = GridSearchCV(model, tuning_params, scoring='roc_auc', cv=cv, n_jobs=-1)

gs.fit(x_train, y_train)
print('Best ROC AUC: %.3f' % gs.best_score_)
print('Best ROC AUC: ', gs.best_params_)

Best ROC AUC: 0.990
Best ROC AUC:  {'C': 0.4}


C:\Users\RJP1F47\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Use lasso and CV to select the features to use and then use regular regression to actually measure the coefficients. Using this as we could see normal regression overfitting

Use lasso/elasticnet findigs now for feature selection if desired

In [67]:
#lasso_fs = Lasso(alpha = gs.best_params_['alpha'])
#lasso_fs.fit(df, y) #I made a change as I forgot to put x and y train in
#lasso_coef = lasso_fs.sparse_coef_
#lasso_coef

In [68]:
## use the lasso coefficients that are not 0 to select those coefficients only from df
#map feature weights to feature names
#dr_features = dict(zip(columns, list(lasso_coef.toarray()[0])))
#dr_list = [i for i,j in dr_features if j > 0.0]

In [74]:
optimal_c = gs.best_params_['C']
type(optimal_c)

float

In [75]:
#model = LogisticRegression(solver='liblinear', class_weight='balanced')
model = LogisticRegression(solver='saga', class_weight='balanced', penalty='l1', C = optimal_c)
model.fit(df,y)

LogisticRegression(C=0.4, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)

In [76]:
reg_model = pd.DataFrame([columns, model.coef_[0]]).sort_values(by=[1], axis=1).T
reg_model = reg_model.reset_index()

In [77]:
#reg_model.iloc[:,0:10]
reg_model.iloc[0:10,:]

,index,0,1
0,145721,Cookies Enabled,-9.79599
1,145506,Cookies_Disabled,-1.63104
2,145538,IF Unable to Assign Device ID - Web,-1.5592
3,145700,Unable_to_Assign_DeviceID_Web,-1.5592
4,155504,True IP Country=united states,-1.16564
5,1449,PID_LCL_AGE_ACC_EMAIL_LT_DAY,-0.9698
6,155426,Smart ID Match Result=Missing,-0.857458
7,145722,Cookies Enabled=Missing,-0.856944
8,145460,2Dim_Good_3Mth_Persona_Gbl,-0.843346
9,155414,OS=Missing,-0.838869


In [78]:
reg_model.iloc[-10:,:]

,index,0,1
155501,11135,_ACC_EMAIL_EXACT_ID_LCL_PAIR_AGE_LT_HR,0.94894
155502,27206,_ACC_EMAIL_LCL_AGE_LT_DY,1.01351
155503,4181,PID_LCL_RLTD_ACC_EMAIL_PASSED_29,1.05312
155504,145379,Level_1_Link_Reject,1.07626
155505,145671,TMXIDPaymentGBL_wk_GT10,1.08573
155506,145284,_TrueIP_FRS_reject_local_high,1.10427
155507,145643,ProbableFraud_2wk,1.21889
155508,890,PID_GBL_RLTD_EXACT_ID_PASSED_27,1.44788
155509,145519,Distinct_IP_Digital_Id:161581022:,2.12159
155510,145626,NON_home_depot_ISP,6.14083


In [79]:
reg_model[(reg_model.index <100) | (reg_model.index > (reg_model.shape[0] -100))].to_csv(r'Fraud Insights Data3.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, pipeline, preprocessing
df = pd.DataFrame({'a':range(12), 'b':[1,2,3,1,2,3,1,2,3,3,1,2], 'c':['a', 'b', 'c']*4, 'd': ['m', 'f']*6})
y = df.a
num = df[['b']]
cat = df[['c', 'd']]
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer(sparse = False)
enc_data = enc.fit_transform(cat .T.to_dict().values())
crat = pd.DataFrame(enc_data,  columns=enc.get_feature_names())
X = pd.concat([crat, num], axis=1)
#cat_columns = ['c=a', 'c=b', 'c=c', 'd=f', 'd=m'] 
cat_columns=list(crat.columns) #replaces line above
cat_indices = np.array([(column in cat_columns) for column in X.columns], dtype = bool)
#numeric_col = ['b']
numeric_col = list(num.columns) #replaces line above
num_indices = np.array([(column in numeric_col) for column in X.columns], dtype = bool)
reg = linear_model.SGDRegressor()
estimator = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
            ('categorical', preprocessing.FunctionTransformer(lambda data: data[:, cat_indices])), 

            #numeric
            ('numeric', pipeline.Pipeline(steps = [
                ('select', preprocessing.FunctionTransformer(lambda data: data[:, num_indices])),
                ('scale', preprocessing.StandardScaler())            
                        ]))
        ])),
    ('model', reg)
    ]
)
estimator.fit(X, y)

clf = GradientBoostingClassifier(n_estimators= 500)
param_dictionary = {"learning_rate": [0.3,0.5], 'max_depth': [1,2,4]}
gs = GridSearchCV(clf, param_dictionary, verbose=3, scoring='roc_auc', cv=5)
gs.fit(x_train, y_train)
print(gs.best_estimator_)
gs.best_score_

In [ ]:
clf = AdaBoostClassifier(n_estimators=100)
param_dictionary = {"learning_rate": [0.3,0.5], 'max_depth': [1,2,4]}
gs = GridSearchCV(clf, param_dictionary, verbose=3, scoring='roc_auc', cv=5)
gs.fit(x_train, y_train)
print(gs.best_estimator_)
gs.best_score_
#scores = cross_val_score(clf, x_train, y_train, scoring='roc_auc', cv=5)
#scores.mean()